# Datenvisualisierung und Exploration

- DatasetReportBuilder



Installation der notwendigen Bibliothen

In [ ]:
pip install hana-ml

In [ ]:
from hana_ml import dataframe

connection = dataframe.ConnectionContext( address = '<Hostname>',
                                         port = '30015',
                                          user = '<User>'
                                           )

In [ ]:
connection.hana_version()

## Erstellen eines HANA Dataframes.
- Quelltable: CHURN (enthält Informationen über Kunden)

In [ ]:
df_churn = connection.table( table = 'CHURN', schema = 'ML_DATA')

In [ ]:
df_churn.head(10).collect()

Verständnisfragen:

- Was ist der Unterschied zwischen HANA-Dataframe und Panadas-Dataframe?
- Was bewirkt der Befehl ``collect()``?

## Dataset Report

In [ ]:
from hana_ml.visualizers.dataset_report import DatasetReportBuilder

datasetReportBuilder = DatasetReportBuilder()
datasetReportBuilder.build( df_churn.cast('EXITED','NVARCHAR(1)'), key = 'CUSTOMERID')
# Ausgabe in HTML-Datei:
#datasetReportBuilder.generate_html_report('Report.html')
datasetReportBuilder.generate_notebook_iframe_report()

Verständnisfragen:

- Welche Informationen stellt der Dataset-Report bereit?
- Wie werden numerische Variablen visualisiert, wie werden kategoriale Variablen visualisiert?
- Welche Auffälligkeiten gibt es in der Datenmenge?

# Cluster-Analysen
- Ausführung des K-Means-Algorithmus
- Interpretation der Ergebnisse

## Ausführung des K-Means-Algorithmus

Filtern der Grundmenge (Tabelle CHURN) auf Kunden mit ``BALANCE > 0``.
Selektion der Spalten für die Cluster-Analyse

In [ ]:
df_churn_subset = df_churn.filter('BALANCE > 0').select(['CUSTOMERID','AGE',
                                                             'BALANCE',
                                                             'ESTIMATEDSALARY',
                                                             'TENURE',
                                                             'GEOGRAPHY',
                                                             'CREDITSCORE',
                                                             'NUMOFPRODUCTS'])

In [ ]:
# Clustering mit K-Means
from hana_ml.algorithms.pal.clustering import KMeans

kmeans = KMeans( n_clusters_min = 4,
                 n_clusters_max = 10,
                 init = 'first_k',
                 max_iter = 100,
                 distance_level = 'Euclidean',
                 accelerated = True,
                 category_weights = 0.5,
                 # normalization = 'min_max'
                  )

df_kmeans_assignment = kmeans.fit_predict(
                                          data = df_churn_subset,
                                          key = 'CUSTOMERID' )
# Cluster.Zuweisung ausgeben
df_kmeans_assignment.head(10).collect()

In [ ]:
# Clusterzentren ausgeben
kmeans.cluster_centers_.collect()


## Interpretation der Ergebnisse

## Verknüpfen der Clusterzuweisung und Quellsätze
- Join zwischen Eingabedaten und Clusterzuweisung
- Auswertung der Attribute nach Cluster mit Boxplot
- Ausgabe eines Streudiagramms

In [ ]:
# Join zwischen Cluster-Zuweisung und Original-Daten
df_cluster_1 = df_kmeans_assignment.rename_columns({'CUSTOMERID' : 'CL_CUSTID'})

df_cust_w_cluster = df_churn_subset.alias('CUST').join(
                                          other = df_cluster_1.alias('CLST'),
                                          condition = 'CUSTOMERID = CL_CUSTID' )

# Entferne überflüssige Spalte CL_CUSTID
df_cust_w_cluster = df_cust_w_cluster.drop('CL_CUSTID')
df_cust_w_cluster.head(10).collect()

In [ ]:
# Box plot nach Cluster
from hana_ml.visualizers.eda import EDAVisualizer
import matplotlib.pyplot as plt

f = plt.figure(figsize=(18,6))

ax1 = f.add_subplot(121)
eda = EDAVisualizer(ax1)


ax, bar_data = eda.box_plot(data= df_cust_w_cluster,
    column = 'ESTIMATEDSALARY',
    groupby = 'CLUSTER_ID',
    outliers = True )

In [ ]:
import plotly.express as px

l_pd_frame = df_cust_w_cluster.collect()

l_pd_frame["CLUSTER_ID_CAT"] = l_pd_frame["CLUSTER_ID"].astype('category')


fig = px.scatter(l_pd_frame,         x = "BALANCE",
                                     y = "ESTIMATEDSALARY",
                                     color = "CLUSTER_ID_CAT",
                                     hover_data  = ['CUSTOMERID','GEOGRAPHY','AGE'] )
fig.show()